In [1]:
# CREATE DATABASE review;
import pandas as pd
import pymysql
import csv
import os

hostName = 'termproject1.ccovdkmpgels.ap-northeast-2.rds.amazonaws.com'
userName = 'uomoy'
password = '#########'
dbName = 'review'

db = pymysql.connect(host=hostName, port=3306, user=userName, passwd=password, db=dbName, charset='utf8')
cursor = db.cursor()
cursor.execute("set names utf8")
db.commit()

C:\Users\JHW\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [30]:
# table restaurants 생성
sql = """CREATE TABLE restaurants (
           id INT UNSIGNED NOT NULL AUTO_INCREMENT,
           restaurant VARCHAR(100) NOT NULL,
           review_num INT NOT NULL,
           PRIMARY KEY(id)
    ) DEFAULT CHARSET=utf8;
    """

cursor.execute(sql)

path_dir = '../YoGiYo-Review-Crawling/reviews'
file_list = os.listdir(path_dir)
restaurant_list = []
for item in file_list:
    restaurant_list.append(item.replace('.csv', ''))
len(restaurant_list)

try:
    for restaurant in restaurant_list:
        df = pd.read_csv('../YoGiYo-Review-Crawling/reviews/' + str(restaurant) + '.csv', engine='python')
        restaurant_sql = "INSERT INTO restaurants (restaurant, review_num) VALUES(\""+str(restaurant)+"\", '"+str(len(df))+"');"
        cursor.execute(restaurant_sql)
finally:
    db.commit()

In [14]:
# table menus 생성

sq = """CREATE TABLE menus (
           id INT UNSIGNED NOT NULL AUTO_INCREMENT,
           restaurant VARCHAR(100) NOT NULL,
           menu VARCHAR(200) NOT NULL,
           PRIMARY KEY(id)
    ) DEFAULT CHARSET=utf8;
    """
cursor.execute(sq)
db.commit()

path_dir = '../YoGiYo-Review-Crawling/reviews'
file_list = os.listdir(path_dir)
restaurant_list = []
for item in file_list:
    restaurant_list.append(item.replace('.csv', ''))

try:
    for restaurant in restaurant_list:
        allMenu = menu_info(review_info(restaurant)[1])[1]
        for menu in allMenu:
            menu_sql = "INSERT INTO menus (restaurant, menu) VALUES(\""+str(restaurant)+"\", \'"+str(menu)+"\');"
            cursor.execute(menu_sql)
finally:
    db.commit()

In [15]:
# table db1, db2, db3 생성
sql1 = """CREATE TABLE menu_analysis (
           id INT UNSIGNED NOT NULL AUTO_INCREMENT,
           restaurant VARCHAR(100) NOT NULL,
           menu VARCHAR(200) NOT NULL,
           order_times INT(10) NOT NULL,
           average FLOAT(4, 2) NOT NULL,
           PRIMARY KEY(id)
    ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
    """
sql2 = """CREATE TABLE menu_img (
           id INT UNSIGNED NOT NULL AUTO_INCREMENT,
           restaurant VARCHAR(100) NOT NULL,
           menu VARCHAR(200) NOT NULL,
           img VARCHAR(100) NOT NULL,
           PRIMARY KEY(id)
    );
    """
sql3 = """CREATE TABLE wordclouds (
           id INT UNSIGNED NOT NULL AUTO_INCREMENT,
           restaurant VARCHAR(100) NOT NULL,
           pn VARCHAR(10) NOT NULL,
           wordcloud MEDIUMTEXT NOT NULL,
           PRIMARY KEY(id)
    );
    """
cursor.execute(sql1)
cursor.execute(sql2)
cursor.execute(sql3)
db.commit()

db1 = pd.read_csv('../YoGiYo-Review-Crawling/processed-reviews/db1.csv', header=None)    # 메뉴별 리뷰횟수 평균별점
db2 = pd.read_csv('../YoGiYo-Review-Crawling/processed-reviews/db2.csv', header=None)     # 메뉴별 이미지
db3 = pd.read_excel('../YoGiYo-Review-Crawling/processed-reviews/db3.xlsx', header=None)   # 음식점별 긍부정 키워드 wordcloud
try:
    for i in range(len(db1)):
        db1_sql = "INSERT INTO menu_analysis (restaurant, menu, order_times, average) VALUES(\""+str(db1.values[i][0])+"\", \""+str(db1.values[i][1])+"\", '"+str(db1.values[i][2])+"', '"+str(db1.values[i][3])+"');"

        cursor.execute(db1_sql)
    print('db1 complete')
    for j in range(len(db2)):
        db2_sql = "INSERT INTO menu_img (restaurant, menu, img) VALUES(\""+str(db2.values[j][0])+"\", \""+str(db2.values[j][1])+"\", '"+str(db2.values[j][2])+"');"
        cursor.execute(db2_sql)
    print('db2 complete')
    for k in range(len(db3)):
        db3_sql = "INSERT INTO wordclouds (restaurant, pn, wordcloud) VALUES(\""+str(db3.values[k][0])+"\", '"+str(db3.values[k][1]) + "', '" +str(db3.values[k][2])+"');"
        cursor.execute(db3_sql)
    print('db3 complete')
finally:
    db.commit()

db1 complete
db2 complete
db3 complete


In [22]:
# AWS Elastic Beanstalk에서 한글 디렉토리를 읽지 못해 '음식점이름'.png -> '음식점 id'.png 로 바꾸는 과정

cursor.execute('select * from restaurants;')
db.commit()
rest_list = list(cursor.fetchall())
pos = os.listdir('../YoGiYo-Review-Crawling/processed-reviews/wordclouds/pos')
neg = os.listdir('../YoGiYo-Review-Crawling/processed-reviews/wordclouds/neg')
for i in pos:
    for j in rest_list:
        if j[1] == i.replace('.png', ''):
            os.rename('../YoGiYo-Review-Crawling/processed-reviews/wordclouds/pos/'+i, '../YoGiYo-Review-Crawling/processed-reviews/wordclouds/pos/'+str(j[0])+'.png')

for i in neg:
    for j in rest_list:
        if j[1] == i.replace('.png', ''):
            os.rename('../YoGiYo-Review-Crawling/processed-reviews/wordclouds/neg/'+i, '../YoGiYo-Review-Crawling/processed-reviews/wordclouds/neg/'+str(j[0])+'.png')

In [26]:
db.close()

Error: Already closed

In [7]:
## crawling.ipynb 에서 정의한 함수

# list of string --> list of list
def string_to_list(list_of_string):
    a = []
    for item in list_of_string:
        i = item.replace('[', '').replace("]", "")
        a.append(i.replace('"', '').replace("'", "").split(','))
    for item in a:
        if '' in item:
            item.remove('')
    return a

# 리뷰 정보 리스트화
def review_info(restaurant_name):
    f = open('../YoGiYo-Review-Crawling/reviews/' + str(restaurant_name) + '.csv', 'r', encoding = 'UTF-8')
    rdr = csv.reader(f)
    l = []
    for line in rdr:
        l.append(line)
    f.close()
    star_point = []
    b = []
    review_text = []
    d = []
    for i in range(len(l)):
        if '[' in l[i][3]:
            index = l[i][3].index('[')
            l[i][3] = l[i][3][:index]
        else:
            continue
    del l[0]
    for item in l:
        star_point.append(item[1])
        b.append(item[2])
        review_text.append(item[3])
        d.append(item[4])
    order_menu = string_to_list(b)
    review_img = string_to_list(d) 
    return star_point, order_menu, review_text, review_img

# 메뉴 정보 리스트화
def menu_info(order_menu):
    a = []
    processed_order_menu = []
    all_menu = []
    review_num = []
    for menu in order_menu:
        empty = []
        for item in menu:
            empty.append(item.strip())
        processed_order_menu.append(empty) # order_menu 에서 각 item 양옆의 공백을 제거한 리스트
    for menu in processed_order_menu:
        for item in menu:
            a.append(item)
    for item in list(set(a)):
        all_menu.append(item.replace(',', '.'))
    for item in all_menu:
        review_num.append(a.count(item))
    
    return processed_order_menu, all_menu, review_num